## Getting Started with rastertools
This document provides an introduction of rastertools. A jupyter-notebook file with the same code is also provided here (can be run, in contrary to this file). 

## Functionalities

+ functions to manipulate rasters, e.g., read, save, clip and tile rasters (`./raster.py`) 
+ extract metadata from raster (`./metadata.py`) 
+ practical tools for converting between grayscale and rgb(a) or coordinates related conversion (`./convert.py`) 
+ Include basic coordinate systems for the Moon and Mars (`./crs.py`) 

## Import of modules

In [4]:
import sys
sys.path.append("/home/nilscp/GIT/")

import geopandas as gpd
import numpy as np
import rasterio as rio

from pathlib import Path
from affine import Affine

import rastertools.raster as raster
import rastertools.metadata as raster_metadata
import rastertools.crs as raster_crs
import rastertools.convert as raster_convert

## Getting Prepared 

*(working directory, download of original raster)*

Let's assume that you work on a Linux or UNIX machine. If this is not the case, I would advice you to install [Git for Windows](https://gitforwindows.org/) on your Windows computer. 

Let's save all of the future beautiful outputs of this tutorial in the temporary directory of your home folder. 

In [5]:
home_p = Path.home()
work_dir= home_p / "tmp" / "rastertools"
raster_dir = (work_dir / "resources" / "raster")
shp_dir = (work_dir / "resources" / "shp")

# Let's define the working directories
work_dir.mkdir(parents=True, exist_ok=True)
raster_dir.mkdir(parents=True, exist_ok=True) 
shp_dir.mkdir(parents=True, exist_ok=True) 

And we can download the original raster and two shapefiles that we will use for this tutorial from my GoogleDrive. I am using the `gdown` library to download the GDrive files. Let's install it quickly within Python. 

In [6]:
!pip install gdown
import gdown


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.8 -m pip install --upgrade pip


We can now download the raster and shapefiles.

In [8]:
url_raster = "https://drive.google.com/uc?id=115Ww5kouD7BO1qDzfdp1MGRuqqVCEoZc"
url_shapefiles= "https://drive.google.com/uc?id=1ln9FXZNEniuJ2y1KLkH8sn9LlVAUTH3M"
gdown.download(url_raster, (raster_dir / "M1221383405.tif").as_posix(), quiet=True)
gdown.download(url_shapefiles, (shp_dir / "shapefiles.zip").as_posix(), quiet=True)
# only work for Linux or UNIX machine (for Windows user, you can unzip the folder manually)
!unzip ~/tmp/rastertools/resources/shp/shapefiles.zip -d ~/tmp/rastertools/resources/shp/ 

Archive:  /home/nilscp/tmp/rastertools/resources/shp/shapefiles.zip
  inflating: /home/nilscp/tmp/rastertools/resources/shp/crater_ROI.cpg  
  inflating: /home/nilscp/tmp/rastertools/resources/shp/crater_ROI.dbf  
  inflating: /home/nilscp/tmp/rastertools/resources/shp/crater_ROI.prj  
  inflating: /home/nilscp/tmp/rastertools/resources/shp/crater_ROI.shp  
  inflating: /home/nilscp/tmp/rastertools/resources/shp/crater_ROI.shx  
  inflating: /home/nilscp/tmp/rastertools/resources/shp/rectangular_ROI.cpg  
  inflating: /home/nilscp/tmp/rastertools/resources/shp/rectangular_ROI.dbf  
  inflating: /home/nilscp/tmp/rastertools/resources/shp/rectangular_ROI.prj  
  inflating: /home/nilscp/tmp/rastertools/resources/shp/rectangular_ROI.shp  
  inflating: /home/nilscp/tmp/rastertools/resources/shp/rectangular_ROI.shx  


Ok, we should be all set now! 

The raster is an image of the surface of the Moon (`M1221383405`), and is actually a mosaic of two Lunar Reconnaissance Orbiter (LRO) Narrow Angle Camera (NAC) images: `M1221383405LE` (https://wms.lroc.asu.edu/lroc/view_lroc/LRO-L-LROC-2-EDR-V1.0/M1221383405LE) and `M1221383405RE` (https://wms.lroc.asu.edu/lroc/view_lroc/LRO-L-LROC-2-EDR-V1.0/M1221383405RE). 

`rectangular_grid` is a polygon shapefile, which is a rectangular polygon roughly centered on the fresh impact crater in the middle of the NAC image. 

`crater_ROI` is a polygon shapefile, which is a circle centered on the fresh impact crater in the middle of the NAC image. 

### Reading a raster

In [11]:
r = raster_dir / "M1221383405.tif"

In order to read a raster, you can use the `read_raster` function:

In [13]:
array = raster.read(r) # to read the whole raster with all bands

<center><img src="../images/image-20230613160540583.png"/></center>

*Figure 1. NAC image M1221383405 (displayed in QGIS)*

But you can include options if needed, such as selecting only the `bands` you are interested in:

In [16]:
array = raster.read(r, bands=[1]) # bands starting from 1, in our case, the example raster has only one band...

You can also choose with the `as_image` flag if you want to have your array loaded with the rasterio (bands, rows, columns) or the image format (rows, columns, bands) . 

In [18]:
# image format
array = raster.read(r, bands=[1], as_image=True) 
print(array.shape)

# rasterio format 
array = raster.read(r, bands=[1], as_image=False) 
print(array.shape)

(55680, 12816, 1)
(1, 55680, 12816)


If you don't want to load the whole raster, you can specify the bounding box of a portion of the image, and only the data within this portion will be loaded. Let's say you are only interested in the area around the very fresh impact crater in the middle of the original raster, and we have a polygon shapefile that constrain the boundary. 

In [19]:
poly = shp_dir / "rectangular_ROI.shp"
gdf_poly = gpd.read_file(poly) # load a rectangular box covering the fresh impact crater in the middle of the image
bounds_poly = list(gdf_poly.bounds.values[0])
array = raster.read(r, bands=[1], bbox=bounds_poly, as_image=True) 
array.shape

(4500, 4194, 1)

If you want to save it as a new raster to avoid the use of the large original raster, which may slow down your computer, you can "clip" your raster. In order to save the new raster, the metadata (see Metadata section at the bottom of this file) of the new raster need to be created. We can use:

In [20]:
original_raster_profile = raster_metadata.get_profile(r)
new_raster_profile = original_raster_profile.copy() 

The width, height and transform metadata need to be updated.

In [21]:
new_raster_profile["transform"]
Affine(0.6339945614856195, 0.0, 10559291.7031,0.0, -0.6339945671695403, -428407.4778)

Affine(0.6339945614856195, 0.0, 10559291.7031,
       0.0, -0.6339945671695403, -428407.4778)

See https://en.wikipedia.org/wiki/Affine_transformation for more info about Affine transformation or write `Affine?`. But long story short, you need to specify the following: 

In [22]:
raster_resolution = raster_metadata.get_resolution(r)[0]
# Affine(raster_resolution, 0.0, xmin, -raster_resolution, ymax) # xmin, ymax corresponds to the top left corner of the image
new_transform = Affine(raster_resolution, 0.0, bounds_poly[0], 0.0, -raster_resolution, bounds_poly[3])

Let's update the metadata:

In [23]:
new_raster_profile.update({
         "width": array.shape[1],
         "height": array.shape[0],
         "transform": new_transform})

### Save the new raster

In [24]:
out_raster = raster_dir / (r.stem + "_bbox_clip1.tif")
raster.save(out_raster, array, new_raster_profile, is_image=True)

<center><img src="../images/image-20230613165613689.png" width="600" height="600" /></center>

*Figure 2. Clipped raster (displayed in QGIS, no scale is provided.)*

NB! Using this workflow, was only for tutorial purpose as it introduces the user to basic functions such as `read` and `save` and the use of metadata-related functions. This pipeline actually introduce some shifts between the original and the new raster because of the coordinate of the top left extent of the polygon shapefile do not fall on the top left edge of a pixel (it can be fixed, but I am not covering that in the tutorial). 

### Clipping

For a correct behavior for the clipping of rasters, please use `clip_from_bbox` or `clip_from_polygon`:

In [25]:
out_raster = raster_dir / (r.stem + "_bbox_clip2.tif")
raster.clip_from_bbox(r, bounds_poly, out_raster)

With a polygon (if we want a different shape than a rectangular bounding box, for example a circle): 

In [26]:
in_polygon = shp_dir / "crater_ROI.shp"
out_raster = raster_dir / (r.stem + "_crater_clip.tif")
raster.clip_from_polygon(r, in_polygon, out_raster)

AttributeError: 'function' object has no attribute 'mask'